# *Group Members*
### Shameen Ghyas CS23002
### Afra Khurram Ansari CS23008
### Amna Ahmed CS23016

# **Necessary libraries**

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import time
import random
import copy
import math
import heapq
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML


# Global stats dictionary: It will be reset by each algorithm
stats = {}


# Helper Functions

In [ ]:
# Helper Functions

# Board representation: list where index = row, value = column
# This design eliminates row conflicts by default

def is_safe(board, row, col):
    """
    Check if placing a queen at (row, col) is safe from attacks.
    Only checks upward since we fill rows top to bottom.

    Args:
        board: Current board configuration
        row: Row position to check
        col: Column position to check

    Returns:
        True if position is safe, False if under attack
    """
    up = row - 1
    while up >= 0:
        # vertically up
        if board[up] == col:
            return False
        # diagonally right or left
        if abs(board[up] - col) == abs(up - row):
            return False
        up -= 1
    return True


def random_configuration(n):
    """
    Generate a random board configuration for search algorithms.
    Places one queen randomly in each row.

    Args:
        n: Size of the board (n x n)

    Returns:
        List of length n with random column positions
    """
    board = []
    for i in range(n):
        board.append(random.randint(0, n - 1))
    return board


def calculate_attacking_pairs(configuration):
    """
    Calculate the number of attacking pairs (conflicts) in current configuration.
    Used as cost function for search algorithms.

    Args:
        configuration: Current board state

    Returns:
        Integer count of queen pairs that attack each other
    """
    count = 0
    n = len(configuration)
    for i in range(n):
        for j in range(i+1, n):
            # check row conflicts
            if configuration[i] == configuration[j]:
                count += 1
            # check diagonal conflicts
            elif abs(configuration[i] - configuration[j]) == abs(i - j):
                count += 1
    return count


class PriorityQueue:
    """
    Priority Queue implementation for A* and Greedy Best-First search.
    Uses heapq for efficient min-heap operations.
    """
    def __init__(self):
        self.queue = []

    def push(self, item, priority):
      # Add item with given priority (lower = higher priority)
      heapq.heappush(self.queue, (priority, item))

    def pop(self):
       # Remove and return item with lowest priority value"""
      priority, item = heapq.heappop(self.queue)
      return priority, item

    def is_empty(self):
      # Check if queue has no elements
      return len(self.queue) == 0


def generate_successors(board):
    """
    Generate next possible board states for search algorithms.
    If board is incomplete, fills next empty position.
    If board is complete, generates all single-queen moves.

    Args:
        board: Current board state (may contain -1 for empty positions)

    Returns:
        List of successor board configurations
    """
    stats['iterations'] += 1
    successors = []
    n = len(board)

    if -1 in board:   # if board not complete, fill next empty row
        row = board.index(-1)
        for col in range(n):
            new_board = board.copy()
            new_board[row] = col
            successors.append(new_board)
            stats['nodes_generated'] += 1
        return successors

    # generate all single-queen moves
    return generate_all_successors(board)


def calculate_heuristic_val(board):
    """
    Calculate heuristic value for informed search (h(n)).
    Counts number of attacking pairs - goal is to reach 0.

    Args:
        board: Current board state (may contain -1 for unplaced queens)

    Returns:
        Number of conflicts (lower is better)
    """
    count = 0
    n = len(board)
    for i in range(n):
        if board[i] == -1:   # Skip unplaced queens
            continue
        for j in range(i + 1, n):
            if board[j] == -1:   # Skip unplaced queens
                continue
            # Check column conflict
            if board[i] == board[j]:
                count += 1
            # Check diagonal conflict
            elif abs(board[i] - board[j]) == abs(i - j):
                count += 1
    return count


def best_successor(current):
    """
    Return the best successor state for algorithm.
    Only returns successor if it has fewer conflicts than current state.

    Args:
        current: Current board configuration

    Returns:
        Best successor board (or current if no improvement found)
    """
    stats['iterations'] += 1
    successors = generate_all_successors(current)
    best = current
    best_value = calculate_attacking_pairs(current)

    for successor in successors:
        successor_value = calculate_attacking_pairs(successor)
        if successor_value < best_value:
            best = successor
            best_value = successor_value
            stats['improvements'] += 1

    return best


def generate_all_successors(configuration):
    """
    Generate all possible successor states by moving one queen.
    Creates n*(n-1) successors for n-queens problem.

    Args:
        configuration: Current complete board state

    Returns:
        List of all boards with one queen moved to different column
    """
    all_successors = []
    n = len(configuration)
    for row in range(n):
        for col in range(n):
            # Only create successor if queen moves to different column
            if configuration[row] != col:
                new_board = configuration.copy()
                new_board[row] = col
                all_successors.append(new_board)
    return all_successors


def generate_random_successor(configuration):
    """
    Generate one random successor.
    Randomly moves one queen to a different column.

    Args:
        configuration: Current board state

    Returns:
        One randomly selected successor board
    """
    successors = generate_all_successors(configuration)
    random_no = random.randint(0, len(successors) - 1)
    return successors[random_no]


def mcv(domains, board):
    """
    Minimum Remaining Values (MRV) heuristic for CSP.
    Selects the unassigned row with the smallest domain (fewest legal values).
    Also known as "most constrained variable" or "fail-first" heuristic.

    Args:
        domains: List of available values for each row
        board: Current board state

    Returns:
        Row index with smallest domain, or -1 if all assigned
    """
    stats['iterations'] += 1
    min_val = len(domains[0]) + 1
    curr_row = -1

    for row in range(len(board)):
        if board[row] == -1:  # Only consider unassigned rows
            domain_size = len(domains[row])
            if domain_size < min_val:
                min_val = domain_size
                curr_row = row

    return curr_row


def lcv(row, domains, board, n):
    """
    Least Constraining Value (LCV) heuristic for CSP.
    Orders domain values by how many options they eliminate for other variables.
    Tries values that leave maximum flexibility for future assignments.

    Args:
        row: Current row being assigned
        domains: Available values for each row
        board: Current board state
        n: Board size

    Returns:
        Sorted list of column values (least constraining first)
    """
    stats['iterations'] += 1

    def count_domains(col):
        eliminated = 0
        for r in range(row + 1, n):
            if board[r] == -1:    # Only check unassigned future rows
                for c in domains[r]:
                    # Count if this column conflicts with the choice
                    if c == col or abs(c - col) == abs(r - row):
                        eliminated += 1
        return eliminated


    # Sort by fewest eliminations (least constraining first)
    return sorted(domains[row], key=count_domains)


# Performance Metrics Function

In [ ]:
# Performance Metrics Calculation Function

def calculate_performance_metrics(algorithm_name, n, initial_state, final_state, runtime, stats_dict, extra_info=None, output_widget=None):
    """
    Universal function to calculate performance metrics for all algorithms

    MANDATORY metrics (returned in dict):
    - Runtime
    - Success Rate
    - Number of Iterations
    - Optimality/Efficiency

    OPTIONAL metrics (printed to output widget):
    - Initial conflicts, Final conflicts, Nodes explored, etc.
    """

    # Calculate conflicts
    initial_conflicts = calculate_attacking_pairs(initial_state)

    if final_state is None or -1 in final_state:
        final_conflicts = -1
    else:
        final_conflicts = calculate_attacking_pairs(final_state)

    # Calculate success rate
    max_conflicts = (n * (n - 1)) // 2
    if final_conflicts >= 0 and max_conflicts > 0:
        success_rate = ((max_conflicts - final_conflicts) / max_conflicts * 100)
    else:
        success_rate = 0.0

    # Determine optimality
    if final_conflicts == 0:
        optimality = "OPTIMAL"
        efficiency = "Solution found with 0 conflicts"
    elif final_conflicts > 0:
        optimality = "SUB-OPTIMAL"
        efficiency = f"{final_conflicts} conflicts remaining"
    else:
        optimality = "NO SOLUTION"
        efficiency = "Algorithm did not find a solution"

    # Get iterations from stats
    iterations = stats_dict.get('iterations', 0)

    # Build mandatory metrics dictionary
    mandatory_metrics = {
        'algorithm': algorithm_name,
        'n': n,
        'runtime': runtime,
        'success_rate': success_rate,
        'iterations': iterations,
        'optimality': optimality,
        'efficiency': efficiency
    }

    # Print optional metrics to widget if provided
    if output_widget:
        with output_widget:
            output_widget.clear_output()
            print(f"\n--- {algorithm_name} Performance Report (N={n}) ---")
            print(f"Runtime: {runtime:.6f} seconds")
            print(f"Success Rate: {success_rate:.2f}%")
            print(f"Iterations: {iterations:,}")
            print(f"Optimality: {optimality}")
            print(f"Efficiency: {efficiency}")
            print(f"\nOptional Details:")
            print(f"  Initial Conflicts: {initial_conflicts}")
            print(f"  Final Conflicts: {final_conflicts if final_conflicts >= 0 else 'N/A'}")
            print(f"  Conflicts Resolved: {initial_conflicts - final_conflicts if final_conflicts >= 0 else 'N/A'}")

            # Print algorithm-specific stats
            if stats_dict:
                print(f"\nAlgorithm-Specific Metrics:")
                for key, value in stats_dict.items():
                    if key != 'iterations':
                        print(f"  {key}: {value:,}" if isinstance(value, int) else f"  {key}: {value}")

            # Print extra info if provided
            if extra_info:
                print(f"\nAdditional Information:")
                for key, value in extra_info.items():
                    print(f"  {key}: {value}")

            print("-" * 50)

    return mandatory_metrics



# Algorithm Implementations


## CSP : Backtracking with Forward Checking

In [ ]:
def forward_check(row, col, domains, n):
    """
    Forward checking to prune domains after placing a queen.
    Removes invalid column values from future rows based on constraints.

    Args:
        row: Row where queen was just placed
        col: Column where queen was just placed
        domains: Current domain values for each row
        n: Board size

    Returns:
        Updated domains if valid, None if any domain becomes empty (failure)
    """
    stats['iterations'] += 1
    stats['forward_checks'] += 1

    new_domains = [d.copy() for d in domains]

    for r in range(row+1, n):
        # Check each value in the domain of future row
        for c in new_domains[r][:]:

            # Remove column if it conflicts (same column or diagonal)
            if c == col or abs(c-col) == abs(r-row):
                new_domains[r].remove(c)

        if not new_domains[r]:
            return None  # forward checking fails

    return new_domains


def backtrack(board, domains, n):
    """
    Backtracking algorithm with MRV and LCV heuristics.
    Recursively tries to place queens while maintaining constraints.

    Args:
        board: Current board state
        domains: Available values for each row
        n: Board size

    Returns:
        List of all valid solutions found from this state
    """
    stats['iterations'] += 1


    # Base case: if board is complete, return it as a solution
    if -1 not in board:
        return [board.copy()]

    # Use MRV heuristic to select most constrained row
    row = mcv(domains, board)
    if row == -1:    # No unassigned rows found
        return []

    # Use LCV heuristic to order domain values (least constraining first)
    lcv_ = lcv(row, domains, board, n)

    solutions = []

    # Try each column value in LCV order
    for col in lcv_:
        if is_safe(board, row, col):
            board[row] = col    # Place queen

            # Perform forward checking to update domains
            new_domains = forward_check(row, col, domains, n)
            if new_domains:

                # Recursively solve remaining board
                solutions.extend(backtrack(board, new_domains, n))
            else:

                # Forward checking failed, this path is invalid
                stats['backtrack'] += 1
            board[row] = -1    # Backtrack: remove queen

    return solutions


def solve_n_queens_backtracking(n):
    """
    Solve N-Queens using CSP backtracking approach.
    Finds all solutions using constraint propagation.

    Args:
        n: Board size (n x n)

    Returns:
        List of all valid board configurations
    """
    initial_domains = [list(range(n)) for _ in range(n)]
    board = [-1] * n    # -1 represents unassigned position
    return backtrack(board, initial_domains, n)

## Local Search: Simulated Annealing

In [ ]:
def n_queens_simulated_annealing(board):
    """
    Simulated Annealing algorithm for N-Queens.
    Probabilistically accepts worse moves to escape local optima.
    Temperature decreases over time, reducing acceptance of worse moves.

    Args:
        board: Initial complete board configuration

    Returns:
        Best board configuration found (may not be optimal)
    """
    current = board
    T = 500    # Initial temperature (high allows more exploration)
    alpha = 0.95    # Cooling rate (temperature multiplier each iteration)
    max_iterations = 5000
    iteration = 0

    while T > 0 and iteration < max_iterations:
        stats['iterations'] += 1
        iteration += 1


        # Goal reached
        if calculate_attacking_pairs(current) == 0:
            break

        neighbour = generate_random_successor(current)

        # Calculate change in energy (conflicts)
        delta_e = calculate_attacking_pairs(current) - calculate_attacking_pairs(neighbour)

        if delta_e > 0:
            # Better solution: always accept
            current = neighbour
        else:
            # Worse solution: accept with probability based on temperature
            probability = random.random()
            # Higher temperature or smaller worsening = higher acceptance chance
            if probability < math.exp(delta_e/T):
                current = neighbour
                stats['accepted_worse'] += 1
            else:
                stats['rejected_moves'] += 1


        # Cool down temperature
        T *= alpha
        stats['temperature_drops'] += 1

    return current

## Local Search: Hill Climbing & Random Restart Hill Climbing

In [ ]:
def n_queens_hill_climbing(board):
    """
    Basic Hill Climbing algorithm for N-Queens.
    Moves to best neighbor until no improvement is possible.
    Can get stuck in local optima or plateaus.

    Args:
        board: Starting complete board configuration

    Returns:
        Board configuration at local optimum or plateau
    """
    current = board

    while True:
        neighbour = best_successor(current)
        # If no improvement possible, stop (local optimum or plateau)
        if calculate_attacking_pairs(neighbour) >= calculate_attacking_pairs(current):
            stats['plateau_hits'] += 1
            return current

        # Move to better neighbor
        current = neighbour


def n_queens_random_restart_hill_climb(board, n):
    """
    Hill Climbing with Random Restarts to overcome local optima.
    Performs multiple hill climbing runs from random starting points.

    Args:
        board: Initial board (unused, generates random starts)
        n: Board size

    Returns:
        Best solution found across all restarts
    """
    MAX_RESTARTS = 100
    best_solution = None

    for i in range(MAX_RESTARTS):
        stats['restarts'] += 1
        # Generate random starting configuration everytime
        current = random_configuration(n)
        result = n_queens_hill_climbing(current)


        # Keep track of best solution found so far
        if best_solution is None or (calculate_attacking_pairs(result) < calculate_attacking_pairs(best_solution)):
            best_solution = result


        # If perfect solution found, stop early
        if calculate_attacking_pairs(best_solution) == 0:
            break

    return best_solution

## Informed Search Algorithm: A* Search

In [ ]:
def n_queens_a_star_search(board):
    """
    A* search algorithm for N-Queens.
    Uses f(n) = g(n) + h(n) where:
    - g(n) = depth (number of queens placed)
    - h(n) = number of attacking pairs (conflicts)
    Guarantees optimal solution if one exists.

    Args:
        board: Initial board (usually empty: all -1s)

    Returns:
        Solution board if found, None otherwise
    """
    pq = PriorityQueue()
    g = 0    # Initial depth (no queens placed yet)

    # Push initial state with f = g + h
    pq.push(board, g + calculate_heuristic_val(board))
    visited = set()    # Track explored states to avoid cycles

    while not pq.is_empty():
        f, current = pq.pop()    # Get state with lowest f value
        stats['nodes_explored'] += 1


        # Convert to tuple for hashing (lists aren't hashable)
        configuration = tuple(current)
        if configuration in visited:
            stats['nodes_revisited'] += 1
            continue
        visited.add(configuration)


        # Extract g value from f
        g = f - calculate_heuristic_val(current)
        # Goal test
        if -1 not in current and calculate_heuristic_val(current) == 0:
            return current


        # Generate all possible next states
        successors = generate_successors(current)
        for successor in successors:
            h = calculate_heuristic_val(successor)
            # Push with f = (g+1) + h (g+1 because we moved one step deeper)
            pq.push(successor, g + 1 + h)

    return None     # No solution found

## Informed Search Algorithm: Greedy Best First Search

In [ ]:
def n_queens_greedy_best(board):
    """
    Greedy Best-First Search for N-Queens.
    Uses only h(n) to guide search (ignores path cost).
    Faster than A* but may not find optimal solution.
    Always expands node with lowest heuristic value (fewest conflicts).

    Args:
        board: Initial board

    Returns:
        Solution board if found, None otherwise
    """
    pq = PriorityQueue()

    # Push initial state with priority = heuristic only
    pq.push(board, calculate_heuristic_val(board))
    visited = set()    # Track explored states

    while not pq.is_empty():
        h, current = pq.pop()   # Get state with lowest heuristic
        stats['nodes_explored'] += 1


        # Convert to tuple for hashing
        configuration = tuple(current)
        if configuration in visited:
            stats['nodes_revisited'] += 1
            continue
        visited.add(configuration)


        # Goal test
        if -1 not in current and calculate_heuristic_val(current) == 0:
            return current


        # Generate all possible next states
        successors = generate_successors(current)

        for successor in successors:
            h = calculate_heuristic_val(successor)
            # Push with priority = heuristic only (greedy approach)
            pq.push(successor, h)

    return None    # No solution found

## Uninformed Search Algorithm: DLS & IDS

In [ ]:
def DLS(board, depth_limit, row, n):
    """
    Depth-Limited Search (recursive helper for IDS).
    Explores up to a specific depth limit.

    Args:
        board: Current board state
        depth_limit: Maximum depth to explore
        row: Current row being filled
        n: Board size

    Returns:
        True if solution found within depth limit, False otherwise
    """
    stats['iterations'] += 1
    stats['nodes_explored'] += 1


    # Base case: all queens placed successfully
    if row == n:
        return True
    # Depth limit reached without completing board
    if row == depth_limit:
        return False

    for col in range(n):
        if is_safe(board, row, col):
            board[row] = col   # Place queen

            # Recursively try to place remaining queens
            if DLS(board, depth_limit, row + 1, n):
                return True   # Solution found
            board[row] = -1   # Backtrack: remove queen
            stats['backtracks'] += 1

    return False     # No solution at this depth


def n_queens_IDS(n):
    """
    Iterative Deepening Search for N-Queens.
    Combines benefits of BFS (completeness) and DFS (space efficiency).
    Gradually increases depth limit until solution found.

    Args:
        n: Board size (n x n)

    Returns:
        Tuple of (solution board, depth where found) or (None, n) if no solution
    """
    for depth in range(1, n + 1):
        stats['depth_levels'] += 1
        board = [-1 for i in range(n)]
        if DLS(board, depth, 0, n):
            return board, depth
    return None, n

# Driver Functions to call algorithms and calculate performance metrics

In [ ]:
# These functions execute each algorithm, track metrics, and return performance data

def run_backtracking(n, output_widget=None):
    """
    Run Backtracking algorithm with Forward Checking, MRV, and LCV heuristics.

    Args:
        n: Board size (n x n)
        output_widget: Optional widget for displaying output in GUI

    Returns:
        Dictionary containing performance metrics, initial state, and final solution
    """
    global stats

    # Initialize statistics tracking for this algorithm
    stats = {'iterations': 0, 'forward_checks': 0, 'backtrack': 0}

    initial_state = random_configuration(n)

    # Measure execution time
    start_time = time.time()
    solutions = solve_n_queens_backtracking(n)
    end_time = time.time()
    runtime = end_time - start_time

    final_state = solutions[0] if solutions else None


    # Algorithm-specific metrics
    extra_info = {
        'Total Solutions Found': len(solutions) if solutions else 0,
        'Forward Checks': stats['forward_checks'],   # Domain pruning operations
        'Backtracks': stats['backtrack']   # Times algorithm backtracked
    }


    # Calculate comprehensive performance metrics
    metrics = calculate_performance_metrics(
        algorithm_name="Backtracking (FC, MRV, LCV)",
        n=n,
        initial_state=initial_state,
        final_state=final_state,
        runtime=runtime,
        stats_dict=stats,
        extra_info=extra_info,
        output_widget=output_widget
    )

    metrics['initial'] = initial_state
    metrics['final'] = final_state if final_state else [-1]*n

    return metrics


def run_simulated_annealing(n, output_widget=None):
    """
    Run Simulated Annealing algorithm for N-Queens.
    Uses probabilistic approach to escape local optima.

    Args:
        n: Board size (n x n)
        output_widget: Optional widget for displaying output in GUI

    Returns:
        Dictionary containing performance metrics, initial state, and final solution
    """
    global stats

    # Initialize statistics specific to Simulated Annealing
    stats = {'iterations': 0, 'temperature_drops': 0, 'accepted_worse': 0, 'rejected_moves': 0}

    initial_state = random_configuration(n)


    # Measure execution time
    start_time = time.time()
    solution = n_queens_simulated_annealing(initial_state.copy())
    end_time = time.time()
    runtime = end_time - start_time



    # Track Simulated Annealing specific behavior
    extra_info = {
        'Temperature Drops': stats['temperature_drops'],
        'Worse Moves Accepted': stats['accepted_worse'],
        'Moves Rejected': stats['rejected_moves']
    }


    # Calculate comprehensive performance metrics
    metrics = calculate_performance_metrics(
        algorithm_name="Simulated Annealing",
        n=n,
        initial_state=initial_state,
        final_state=solution,
        runtime=runtime,
        stats_dict=stats,
        extra_info=extra_info,
        output_widget=output_widget
    )

    metrics['initial'] = initial_state
    metrics['final'] = solution

    return metrics


def run_hill_climbing(n, output_widget=None):
    """
    Run Random-Restart Hill Climbing algorithm.
    Performs multiple hill climbing runs to overcome local optima.

    Args:
        n: Board size (n x n)
        output_widget: Optional widget for displaying output in GUI

    Returns:
        Dictionary containing performance metrics, initial state, and best solution found
    """
    global stats

    # Initialize statistics for hill climbing with restarts
    stats = {'iterations': 0, 'restarts': 0, 'plateau_hits': 0, 'improvements': 0}

    initial_state = random_configuration(n)

    # Measure execution time
    start_time = time.time()
    solution = n_queens_random_restart_hill_climb(initial_state.copy(), n)
    end_time = time.time()
    runtime = end_time - start_time


    # Track hill climbing specific behavior
    extra_info = {
        'Total Restarts': stats['restarts'],
        'Plateau Hits': stats['plateau_hits'],   # Times algorithm got stuck
        'Improvements': stats['improvements']
    }


    # Calculate comprehensive performance metrics
    metrics = calculate_performance_metrics(
        algorithm_name="Random-Restart Hill Climbing",
        n=n,
        initial_state=initial_state,
        final_state=solution,
        runtime=runtime,
        stats_dict=stats,
        extra_info=extra_info,
        output_widget=output_widget
    )

    metrics['initial'] = initial_state
    metrics['final'] = solution

    return metrics


def run_a_star(n, output_widget=None):
    """
    Run A* Search algorithm for N-Queens.
    Uses f(n) = g(n) + h(n) to find optimal solution.

    Args:
        n: Board size (n x n)
        output_widget: Optional widget for displaying output in GUI

    Returns:
        Dictionary containing performance metrics, initial state, and optimal solution
    """
    global stats

    # Initialize statistics for informed search
    stats = {'iterations': 0, 'nodes_explored': 0, 'nodes_generated': 0, 'nodes_revisited': 0}

    initial_state = random_configuration(n)

    # Measure execution time
    start_time = time.time()
    solution = n_queens_a_star_search(initial_state.copy())
    end_time = time.time()
    runtime = end_time - start_time

    # Track search tree exploration metrics
    extra_info = {
        'Nodes Explored': stats['nodes_explored'],  # Unique states examined
        'Nodes Generated': stats['nodes_generated'],  # Total successor states created
        'Nodes Revisited': stats['nodes_revisited']
    }


    # Calculate comprehensive performance metrics
    metrics = calculate_performance_metrics(
        algorithm_name="A* Search",
        n=n,
        initial_state=initial_state,
        final_state=solution,
        runtime=runtime,
        stats_dict=stats,
        extra_info=extra_info,
        output_widget=output_widget
    )

    metrics['initial'] = initial_state
    metrics['final'] = solution if solution else [-1]*n

    return metrics


def run_greedy(n, output_widget=None):
    """
    Run Greedy Best-First Search algorithm for N-Queens.
    Uses only heuristic h(n) to guide search (faster but not optimal).

    Args:
        n: Board size (n x n)
        output_widget: Optional widget for displaying output in GUI

    Returns:
        Dictionary containing performance metrics, initial state, and solution found
    """
    global stats

    # Initialize statistics for greedy search
    stats = {'iterations': 0, 'nodes_explored': 0, 'nodes_generated': 0, 'nodes_revisited': 0}

    initial_state = random_configuration(n)

    # Measure execution time
    start_time = time.time()
    solution = n_queens_greedy_best(initial_state.copy())
    end_time = time.time()
    runtime = end_time - start_time

    # Track search tree exploration metrics
    extra_info = {
        'Nodes Explored': stats['nodes_explored'],
        'Nodes Generated': stats['nodes_generated'],
        'Nodes Revisited': stats['nodes_revisited']
    }


    # Calculate comprehensive performance metrics
    metrics = calculate_performance_metrics(
        algorithm_name="Greedy Best-First Search",
        n=n,
        initial_state=initial_state,
        final_state=solution,
        runtime=runtime,
        stats_dict=stats,
        extra_info=extra_info,
        output_widget=output_widget
    )

    metrics['initial'] = initial_state
    metrics['final'] = solution if solution else [-1]*n

    return metrics


def run_ids(n, output_widget=None):
    """
    Run Iterative Deepening Search algorithm for N-Queens.
    Combines completeness of BFS with space efficiency of DFS.

    Args:
        n: Board size (n x n)
        output_widget: Optional widget for displaying output in GUI

    Returns:
        Dictionary containing performance metrics, initial state, and solution found
    """
    global stats

     # Initialize statistics for IDS
    stats = {'iterations': 0, 'depth_levels': 0, 'backtracks': 0, 'nodes_explored': 0}

    initial_state = random_configuration(n)

    # Measure execution time
    start_time = time.time()
    solution, depth = n_queens_IDS(n)
    end_time = time.time()
    runtime = end_time - start_time


    # Track IDS specific behavior
    extra_info = {
        'Depth Levels Explored': stats['depth_levels'],   # Number of depth limits tried
        'Solution Found at Depth': depth if solution else 'N/A',
        'Nodes Explored': stats['nodes_explored'],   # Total nodes examined
        'Backtracks': stats['backtracks']   # Times algorithm backtracked
    }


    # Calculate comprehensive performance metrics
    metrics = calculate_performance_metrics(
        algorithm_name="Iterative Deepening Search",
        n=n,
        initial_state=initial_state,
        final_state=solution,
        runtime=runtime,
        stats_dict=stats,
        extra_info=extra_info,
        output_widget=output_widget
    )

    metrics['initial'] = initial_state
    metrics['final'] = solution if solution else [-1]*n

    return metrics

# Interactive UI with Comparison

In [ ]:
def plot_board(configuration, ax=None, title=None):
    """
    Generate chessboard-style board visualization with queens placed.
    Uses alternating light/dark squares and gold queen symbols.

    Args:
        configuration: List where index=row, value=column position of queen
        ax: Matplotlib axes object (creates new if None)
        title: Optional title for the board

    Returns:
        Tuple of (figure, axes) objects
    """
    n = len(configuration)
    if ax is None:
        fig, ax = plt.subplots(figsize=(3, 3))
    else:
        fig = ax.figure


    # Define chessboard colors
    light_color = "#E8E8E8"   # Light gray for white squares
    dark_color = "#4A4A4A"    # Dark gray for black squares


    # Draw chessboard squares
    for r in range(n):
        for c in range(n):
            # Alternate colors based on position
            color = light_color if (r + c) % 2 == 0 else dark_color
            ax.add_patch(plt.Rectangle((c - 0.5, r - 0.5), 1, 1, facecolor=color, edgecolor='black', linewidth=0.5))


    # Place queens on the board
    for r, c in enumerate(configuration):
        # Skip empty positions (-1 or None)
        if c is None or c == -1:
            continue
        # Draw queen symbol at (column, row)
        ax.text(c, r, "♛", fontsize=28, ha='center', va='center',
                color="#D4AF37", fontweight='bold', fontname='DejaVu Sans')

    # Configure axes
    ax.set_xticks(range(n))
    ax.set_yticks(range(n))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xlim(-0.5, n - 0.5)
    ax.set_ylim(n - 0.5, -0.5)
    ax.set_aspect('equal')

    if title:
        ax.set_title(title, fontsize=12, pad=5, color="#333333")

    return fig, ax


# Algorithm mapping
ALGO_RUNNERS = {
    'Backtracking (FC, MRV, LCV)': run_backtracking,
    'Simulated Annealing': run_simulated_annealing,
    'Random-Restart Hill Climbing': run_hill_climbing,
    'A* Search': run_a_star,
    'Greedy Best-First Search': run_greedy,
    'Iterative Deepening Search': run_ids
}

# UI Widgets

# Dropdown to select which algorithm to run
algo_dropdown = widgets.Dropdown(options=list(ALGO_RUNNERS.keys()), description='Algorithm:')

# Input field for board size (N)
n_input = widgets.IntText(value=8, description='N:')

# Button to execute selected algorithm
run_button = widgets.Button(description='Run Algorithm', button_style='success')  # Green button

# Dropdown to select second algorithm for comparison
compare_dropdown = widgets.Dropdown(options=[], description='Compare With:')

# Button to trigger comparison
compare_button = widgets.Button(description='Compare', button_style='info')

# Output widget for status messages
status_out = widgets.Output(layout={'border': '1px solid black'})

# Output widget for displaying board visualizations
display_out = widgets.Output()

# Output widget for displaying comparison results
compare_out = widgets.Output()


# Dictionary to store metrics from each algorithm run
# Key: algorithm name, Value: metrics dictionary
results = {}

# Callbacks
def on_run_clicked(b):
    """
    Callback function when 'Run Algorithm' button is clicked.
    Validates input, runs selected algorithm, displays results.

    Args:
        b: Button widget (automatically passed by event handler)
    """

    # Display status messages in status_out widget
    with status_out:
        status_out.clear_output()
        n_value = n_input.value

        # Validate board size
        if n_value < 4:
            print("Error: Number of queens must be 4 or greater.")
            return
        if n_value > 25:
            print("Error: Maximum allowed N is 25.")
            return
        print(f"Running {algo_dropdown.value} for N = {n_value} -- please wait...")

    runner = ALGO_RUNNERS[algo_dropdown.value]
    try:
        metrics = runner(n_value, status_out)
    except Exception as e:
      # Handle any errors during algorithm execution
        with status_out:
            status_out.clear_output()
            print(f"Error while running algorithm: {e}")
        return


    # Store results for later comparison
    results[algo_dropdown.value] = metrics

    # Update compare dropdown
    compare_dropdown.options = list(results.keys())


    # Display initial and final board states
    with display_out:
        display_out.clear_output()

        # Create side-by-side subplots
        fig, axes = plt.subplots(1, 2, figsize=(7, 3.5))


        # Plot initial state
        init_conf = metrics['initial']
        if init_conf is None:
            init_conf = [-1] * metrics['n']
        plot_board(init_conf, ax=axes[0], title='Initial State')


        # Plot final solution state
        final_conf = metrics['final']
        if final_conf is None:
            final_conf = [-1] * metrics['n']
        plot_board(final_conf, ax=axes[1], title='Final State')

        plt.tight_layout()
        plt.show()


def on_compare_clicked(b):
    """
    Callback function when 'Compare' button is clicked.
    Compares two algorithm results with visualizations and statistics.

    Args:
        b: Button widget (automatically passed by event handler)
    """
    with compare_out:
        compare_out.clear_output()


        # Check if any algorithms have been run
        if compare_dropdown.value is None:
            print("No algorithms available for comparison. Run at least one algorithm first.")
            return

        a1_name = algo_dropdown.value   # Currently selected algorithm
        a2_name = compare_dropdown.value   # Algorithm to compare with



        # Validate both algorithms have been run
        if a1_name not in results:
            print(f"First run '{a1_name}' before comparing.")
            return
        if a2_name not in results:
            print(f"'{a2_name}' has not been run yet. Run it first to compare.")
            return


        # Get metrics for both algorithms
        m1 = results[a1_name]
        m2 = results[a2_name]


        # Ensure both were run with same board size
        if m1['n'] != m2['n']:
            print(f"Cannot compare '{a1_name}' (N={m1['n']}) with '{a2_name}' (N={m2['n']}).")
            print("Please run both algorithms with the same N value.")
            return

        # Prepare comparison data
        labels = ['Runtime (s)', 'Success Rate (%)', 'Iterations', 'Optimality Score']

        # Convert optimality to numeric score
        def optimality_to_score(opt):
            if opt == "OPTIMAL":
                return 100
            elif opt == "SUB-OPTIMAL":
                return 50
            else:
                return 0


        # Extract metrics for both algorithms
        v1 = [m1['runtime'], m1['success_rate'], m1['iterations'], optimality_to_score(m1['optimality'])]
        v2 = [m2['runtime'], m2['success_rate'], m2['iterations'], optimality_to_score(m2['optimality'])]


        ### Create comparison bar charts

        # Create 2x2 subplot - SMALLER SIZE
        fig, axes = plt.subplots(2, 2, figsize=(9, 5))
        axes = axes.flatten()

        colors1 = 'steelblue'
        colors2 = 'orange'

        # Create bar chart for each metric
        for idx, (label, val1, val2) in enumerate(zip(labels, v1, v2)):
            ax = axes[idx]
            bars = ax.bar([a1_name, a2_name], [val1, val2], color=[colors1, colors2])
            # Configure axes
            ax.set_ylabel(label, fontsize=9)
            ax.set_title(f'{label} Comparison', fontsize=10)
            ax.tick_params(axis='x', rotation=15, labelsize=8)
            ax.tick_params(axis='y', labelsize=8)

            # Add value labels
            for bar in bars:
                h = bar.get_height()
                if label == 'Iterations':
                    label_text = f'{int(h):,}'
                elif label == 'Optimality Score':
                    label_text = f'{int(h)}'
                else:
                    label_text = f'{h:.4f}'   # 4 decimal places

                # Place label above bar
                ax.annotate(label_text,
                           xy=(bar.get_x() + bar.get_width() / 2, h),
                           xytext=(0, 3), textcoords="offset points",
                           ha='center', va='bottom', fontsize=8)

        plt.suptitle(f'Comparison: {a1_name} vs {a2_name} (N={m1["n"]})',
                     fontsize=12, fontweight='bold')
        plt.tight_layout()
        plt.show()

        # Print detailed comparison
        print(f"\nDetailed Comparison (N={m1['n']}):")
        print(f"\n{a1_name}:")
        print(f"  Runtime: {m1['runtime']:.6f}s")
        print(f"  Success Rate: {m1['success_rate']:.2f}%")
        print(f"  Iterations: {m1['iterations']:,}")
        print(f"  Optimality: {m1['optimality']}")
        print(f"  Efficiency: {m1['efficiency']}")

        print(f"\n{a2_name}:")
        print(f"  Runtime: {m2['runtime']:.6f}s")
        print(f"  Success Rate: {m2['success_rate']:.2f}%")
        print(f"  Iterations: {m2['iterations']:,}")
        print(f"  Optimality: {m2['optimality']}")
        print(f"  Efficiency: {m2['efficiency']}")


# Connect callbacks to buttons
run_button.on_click(on_run_clicked)
compare_button.on_click(on_compare_clicked)

# Layout

# Horizontal box for main controls (algorithm selector, N input, run button)
controls = widgets.HBox([algo_dropdown, n_input, run_button])
# Horizontal box for comparison controls
compare_row = widgets.HBox([compare_dropdown, compare_button])
# Vertical box containing all UI elements
ui = widgets.VBox([
    widgets.HTML("<h3>N-Queens Algorithm</h3>"),
    controls,
    status_out,
    display_out,
    widgets.HTML("<hr><b>Compare with previously run algorithms:</b>"),
    compare_row,
    compare_out
])


# Display the complete UI
display(ui)